<a href="https://colab.research.google.com/github/RogerHeederer/PaperReviewCode/blob/main/S2S_with_NeuralNetwork_NIPS2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sequence to Sequence Learning with Neural Network (NIPS,2014)** 실습

Ref : https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Sequence_to_Sequence_with_LSTM_Tutorial.ipynb

In [1]:
#데이터 전처리 - spaCy 라이브러리 사용
# : 문장의 토큰화 및 태깅 등의 전처리 기능을 제공

%%capture
!python -m spacy download en
!python -m spacy download de

In [2]:
import spacy

spacy_en = spacy.load('en') #영어 토큰화
spacy_de = spacy.load('de') #독일어 토큰화

In [3]:
tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [4]:
#독일어 토큰화 + 단어의 순서를 뒤집는 함수
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)][::-1]

#영어 토큰화 함수 정의
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

In [5]:
#필드 라이브러리를 이용해서 데이터 셋에 대한 구체적인 전처리를 명시
# src: 독일어, tar: 영어
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True)

In [6]:
from torchtext.datasets import Multi30k # 대표적 영어-독어 번역 데이터셋

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields=(SRC, TRG))

training.tar.gz:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 5.78MB/s]
validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.37MB/s]

downloading validation.tar.gz
downloading mmt_task1_test2016.tar.gz



mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.30MB/s]


In [7]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(valid dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(valid dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(valid dataset) 크기: 1014개
테스트 데이터셋(valid dataset) 크기: 1000개


In [8]:
#학습 데이터 중 하나를 샘플로 봐보자
print(vars(train_dataset.examples[30])['src']) #src 독어
print(vars(train_dataset.examples[30])['trg']) #trg 영어

['.', 'steht', 'urinal', 'einem', 'an', 'kaffee', 'tasse', 'einer', 'mit', 'der', ',', 'mann', 'ein']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [9]:
#필드 객체의 빌드 보캅 메서드를 이용해서 영어와 독어의 단어 사전을 생성
#최소 2번 이상 빈도를 가진 단어만을 선책

SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"SRC 보카 사이즈 len(SRC): {len(SRC.vocab)}")
print(f"TRG 보카 사이즈 len(TRG): {len(TRG.vocab)}")
print("이게 각각의 원핫 인코딩 차원수가 됨")

SRC 보카 사이즈 len(SRC): 7855
TRG 보카 사이즈 len(TRG): 5893
이게 각각의 원핫 인코딩 차원수가 됨


In [10]:
#build_vocab으로 만든 요소들 살펴보기 .stoi
print(TRG.vocab.stoi["abcbcbcd"]) # 없는 단어:0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


한 문장에 포함된 단어가 연속적으로 LSTM에 입력된다. 그래서!

하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하면, max 길이를 과하게 잡을 필요가 없어 패딩 처리하는 연산을 아낄 수 있다.

이를 위해 bucketiterator 사용한다

배치크기는 128

In [11]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

#일반적인 데이터 로더의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device)

In [12]:
train_iterator

In [13]:
for i, batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg
  
  print(f"첫 번째 배치 크기: {src.shape}")

  for i in range(src.shape[0]): #현재 배치에 있는 하나의 문장에 포함된 정보 출력
    print(f"인덱스 {i}, src[{i}][0] = {src[i][0].item()}")
  #첫 번째 배치만 확인
  break

첫 번째 배치 크기: torch.Size([29, 128])
인덱스 0, src[0][0] = 2
인덱스 1, src[1][0] = 4
인덱스 2, src[2][0] = 179
인덱스 3, src[3][0] = 7808
인덱스 4, src[4][0] = 44
인덱스 5, src[5][0] = 110
인덱스 6, src[6][0] = 15
인덱스 7, src[7][0] = 7
인덱스 8, src[8][0] = 17
인덱스 9, src[9][0] = 9
인덱스 10, src[10][0] = 3074
인덱스 11, src[11][0] = 11
인덱스 12, src[12][0] = 241
인덱스 13, src[13][0] = 3
인덱스 14, src[14][0] = 1
인덱스 15, src[15][0] = 1
인덱스 16, src[16][0] = 1
인덱스 17, src[17][0] = 1
인덱스 18, src[18][0] = 1
인덱스 19, src[19][0] = 1
인덱스 20, src[20][0] = 1
인덱스 21, src[21][0] = 1
인덱스 22, src[22][0] = 1
인덱스 23, src[23][0] = 1
인덱스 24, src[24][0] = 1
인덱스 25, src[25][0] = 1
인덱스 26, src[26][0] = 1
인덱스 27, src[27][0] = 1
인덱스 28, src[28][0] = 1


In [15]:
src[28][127] #src[단어갯수][배치사이즈]

tensor(1, device='cuda:0')

**인코더 아키텍쳐**

주어진 소스 문장을 context vector로 인코딩 함

LSTM 레이어는 히든 스테이트와 셀 스테이트를 반환

하이퍼 파라미터는 다음과 같다

* input_dim: 하나의 단어에 대한 원핫 인코딩 차원
* embed_dim: 임베딩(embedding) 차원
* hidden_dim: 히든 상태(hidden state) 차원
* n_layers: RNN 레이어의 개수
* dropout_ratio: 드롭아웃(dropout) 비율

In [16]:
import torch.nn as nn

#인코더 아키텍쳐 정의
class Encoder(nn.Module):
  def __init__(self, input_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
    super().__init__()

    # 임베딩(embedding)은 원핫 인코딩을 특정 차원의 임베딩으로 매핑시켜 주는 레이어
    self.embedding = nn.Embedding(input_dim, embed_dim)

    # LSTM 레이어
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout = dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

  # 소스 문장을 입력으로 받아 문맥 벡터로 변환
  def forward(self, src): #src : [단어 개수, 배치크기] : 각 단어의 인덱스 정보
    
    # embedded: [단어개수, 배치크기, 임베딩 차원]
    embedded = self.dropout(self.embedding(src))

    # outputs: [단어 개수, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
    # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
    # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
    outputs, (hidden, cell) = self.rnn(embedded)

    return hidden, cell # 문맥 백터 반환

디코더 아키텍쳐

주어진 문맥 벡터를 타겟 문장으로 디코딩

LSTM은 hidden state와 cell state를 반환

하이퍼 파라미터

* input_dim: 하나의 단어에 대한 원핫 인코딩 차원
* embed_dim: 임베딩(embedding) 차원
* hidden_dim: 히든 상태(hidden state) 차원
* n_layers: RNN 레이어의 개수
* dropout_ratio: 드롭아웃(dropout) 비율

In [37]:
#디코더 아키텍쳐 정의
class Decoder(nn.Module):
  def __init__(self, output_dim, embed_dim, hidden_dim, n_layers, dropout_ratio):
    super().__init__()

    #임베딩은 원핫 인코딩 을 특정 차원의 임베딩으로 매핑시키는 레이어
    self.embedding = nn.Embedding(output_dim, embed_dim)

    #LSTM 레이어
    self.hidden_dim = hidden_dim
    self.n_layers = n_layers
    self.rnn = nn.LSTM(embed_dim, hidden_dim, n_layers, dropout=dropout_ratio)

    #FullyConnected 레이어 (인코더와 구조적으로 다른 부분임)
    self.output_dim = output_dim
    self.fc_out = nn.Linear(hidden_dim, output_dim)
    self.dropout = nn.Dropout(dropout_ratio)

    #디코더는 현재까지 출력된 문장에 대한 정보를 입력으로 받아 타겟 문장 반환
  def forward(self, input, hidden, cell):

      # input:[배치 크기] / 디코딩 할 때는 단어를 1개씩 번역함. 그래서 단어 개수는 항상 1개이도록 구현
      # hidden: [레이어 개수, 배치 크기, 히든 차원]
      # cell = context -> [레이어 개수, 배치 크기, 히든 차원]
      input = input.unsqueeze(0) # [1, 배치크기]
      embedded = self.dropout(self.embedding(input)) #embedded: [단어개수, 배치크기, 임베딩 차원]
      output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
      # output: [단어 개수 = 1, 배치 크기, 히든 차원]: 현재 단어의 출력 정보
      # hidden: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보
      # cell: [레이어 개수, 배치 크기, 히든 차원]: 현재까지의 모든 단어의 정보

      prediction = self.fc_out(output.squeeze(0))
      
      return prediction, hidden, cell # 현재 출력 단어, 현재까지의 모든 단어 정보, 현재까지의 모든 단어 정보

Seq2Seq 아키텍쳐

인코더 : 주어진 소스 문장을 문맥 벡터로 인코딩

디코더 : 주어진 문맥 벡터를 타겟 문장으로 디코딩

단, 디코더는 한 단어씩 넣어서 한 번씩 결과를 구한다.

Teacher forcing : 디코더의 예측을 다음 입력으로 사용하지 않고, 실제 목표 출력(그라운 트룻)을 다음 입력으로 사용하는 기법



In [28]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    # 학습할 때는 완전한 형태의 소스 문장, 타겟 문장, teacher_forcing_ratio를 넣기
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [단어 개수, 배치 크기]
        # trg: [단어 개수, 배치 크기]
        # 먼저 인코더를 거쳐 문맥 벡터(context vector)를 추출
        hidden, cell = self.encoder(src)

        # 디코더(decoder)의 최종 결과를 담을 텐서 객체 만들기
        trg_len = trg.shape[0] # 단어 개수
        batch_size = trg.shape[1] # 배치 크기
        trg_vocab_size = self.decoder.output_dim # 출력 차원
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # 첫 번째 입력은 항상 <sos> 토큰
        input = trg[0, :]

        # 타겟 단어의 개수만큼 반복하여 디코더에 포워딩(forwarding)
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output # FC를 거쳐서 나온 현재의 출력 단어 정보
            top1 = output.argmax(1) # 가장 확률이 높은 단어의 인덱스 추출

            # teacher_forcing_ratio: 학습할 때 실제 목표 출력(ground-truth)을 사용하는 비율
            teacher_force = random.random() < teacher_forcing_ratio
            input = trg[t] if teacher_force else top1 # 현재의 출력 결과를 다음 입력에서 넣기
        
        return outputs

In [29]:
# 학습 하이퍼파라미터 설정 및 모델 초기화
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENCODER_EMBED_DIM = 256
DECODER_EMBED_DIM = 256
HIDDEN_DIM = 512
N_LAYERS = 2
ENC_DROPOUT_RATIO = 0.5
DEC_DROPOUT_RATIO = 0.5

In [30]:
# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, ENCODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT_RATIO)
dec = Decoder(OUTPUT_DIM, DECODER_EMBED_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT_RATIO)

# Seq2Seq 객체 선언
model = Seq2Seq(enc, dec, device).to(device)

In [31]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

학습 및 평가

In [32]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
optimizer = optim.Adam(model.parameters())

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [33]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()

        output = model(src, trg)
        # output: [출력 단어 개수, 배치 크기, 출력 차원]
        output_dim = output.shape[-1]
        
        # 출력 단어의 인덱스 0은 사용하지 않음
        output = output[1:].view(-1, output_dim)
        # output = [(출력 단어의 개수 - 1) * batch size, output dim]
        trg = trg[1:].view(-1)
        # trg = [(타겟 단어의 개수 - 1) * batch size]
        
        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산
        
        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        # 파라미터 업데이트
        optimizer.step()
        
        # 전체 손실 값 계산
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [34]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0
    
    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 평가할 때 teacher forcing는 사용하지 않음
            output = model(src, trg, 0)
            # output: [출력 단어 개수, 배치 크기, 출력 차원]
            output_dim = output.shape[-1]
            
            # 출력 단어의 인덱스 0은 사용하지 않음
            output = output[1:].view(-1, output_dim)
            # output = [(출력 단어의 개수 - 1) * batch size, output dim]
            trg = trg[1:].view(-1)
            # trg = [(타겟 단어의 개수 - 1) * batch size]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [35]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [38]:
import time
import math
import random

N_EPOCHS = 20
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 24s
	Train Loss: 4.451 | Train PPL: 85.712
	Validation Loss: 4.774 | Validation PPL: 118.406
Epoch: 02 | Time: 0m 24s
	Train Loss: 4.155 | Train PPL: 63.758
	Validation Loss: 4.598 | Validation PPL: 99.304
Epoch: 03 | Time: 0m 24s
	Train Loss: 3.958 | Train PPL: 52.369
	Validation Loss: 4.375 | Validation PPL: 79.463
Epoch: 04 | Time: 0m 25s
	Train Loss: 3.769 | Train PPL: 43.347
	Validation Loss: 4.324 | Validation PPL: 75.497
Epoch: 05 | Time: 0m 24s
	Train Loss: 3.623 | Train PPL: 37.440
	Validation Loss: 4.186 | Validation PPL: 65.791
Epoch: 06 | Time: 0m 24s
	Train Loss: 3.436 | Train PPL: 31.073
	Validation Loss: 4.107 | Validation PPL: 60.787
Epoch: 07 | Time: 0m 24s
	Train Loss: 3.309 | Train PPL: 27.351
	Validation Loss: 3.999 | Validation PPL: 54.554
Epoch: 08 | Time: 0m 24s
	Train Loss: 3.158 | Train PPL: 23.525
	Validation Loss: 3.964 | Validation PPL: 52.684
Epoch: 09 | Time: 0m 24s
	Train Loss: 3.054 | Train PPL: 21.198
	Validation Loss: 3.823 | Valid

In [ ]:
#학습된 모델 저장
from google.colab import files
files.download('seq2seq.pt')

In [39]:
#모델 최종 테스트 결과 확인
model.load_state_dict(torch.load('seq2seq.pt'))
test_loss = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

Test Loss: 3.620 | Test PPL: 37.349


나의 데이터로 모델 사용해보기

In [43]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더(endocer)에 소스 문장을 넣어 문맥 벡터(context vector) 계산
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        # 이전에 출력한 단어가 현재 단어로 입력될 수 있도록
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(trg_tensor, hidden, cell)

        pred_token = output.argmax(1).item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:]

In [45]:
example_idx = 11

src = vars(test_dataset.examples[example_idx])['src']
trg = vars(test_dataset.examples[example_idx])['trg']

print(f'소스 문장: {src}')
print(f'타겟 문장: {trg}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['.', 'sind', 'luft', 'der', 'in', 'noch', 'immer', 'hände', 'seine', 'während', ',', 'trifft', 'nicht', 'ball', 'den', 'mann', 'ein', 'wobei', ',', 'spielen', 'volleyball', 'die', ',', 'männer']
타겟 문장: ['men', 'playing', 'volleyball', ',', 'with', 'one', 'player', 'missing', 'the', 'ball', 'but', 'hands', 'still', 'in', 'the', 'air', '.']
전체 소스 토큰: ['<sos>', '.', 'sind', 'luft', 'der', 'in', 'noch', 'immer', 'hände', 'seine', 'während', ',', 'trifft', 'nicht', 'ball', 'den', 'mann', 'ein', 'wobei', ',', 'spielen', 'volleyball', 'die', ',', 'männer', '<eos>']
소스 문장 인덱스: [2, 4, 84, 90, 15, 7, 1405, 6439, 363, 139, 35, 9, 2035, 482, 92, 33, 13, 5, 601, 9, 57, 669, 17, 9, 30, 3]
모델 출력 결과: two men are playing , one is the the the the the the air , his hands in the air . <eos>


In [47]:
src = tokenize_de("haben Sie einen guten Tag") #have a good day

print(f'소스 문장: {src}')
print("모델 출력 결과:", " ".join(translate_sentence(src, SRC, TRG, model, device)))

소스 문장: ['Tag', 'guten', 'einen', 'Sie', 'haben']
전체 소스 토큰: ['<sos>', 'tag', 'guten', 'einen', 'sie', 'haben', '<eos>']
소스 문장 인덱스: [2, 200, 3799, 19, 99, 307, 3]
모델 출력 결과: <unk> enjoy a day day . <eos>
